In [1]:
# force geopandas to use shapely instead of pygeos
import os
os.environ['USE_PYGEOS'] = '0'

import numpy as np
import pandas as pd
import plotly.express as px
from shapely import wkt
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [2]:
meters = pd.read_csv("../data/Parking_Meters.csv")

In [3]:
meters['shape'] = meters['shape'].apply(wkt.loads)
meters = gpd.GeoDataFrame(meters, geometry='shape')
meters.crs = 'EPSG:4326'

street_df = pd.read_csv('../data/street_sweeping.csv')
street_df = street_df[~street_df.Line.isnull()]
street_df.loc[:, 'Line'] = street_df.Line.apply(wkt.loads)
street_df = gpd.GeoDataFrame(street_df, geometry='Line')
street_df.crs = 'EPSG:4326'

unique_streets = street_df.drop_duplicates(subset='Line')

In [4]:
joined = gpd.sjoin_nearest(meters, unique_streets, distance_col='dist', how='left')
joined.shape

/Users/tessa/opt/anaconda3/envs/stat-222/lib/python3.9/site-packages/geopandas/array.py:344: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


(35645, 63)

In [5]:
# filter to only include active meters

active = meters[(meters["ACTIVE_METER_FLAG"] == "M") | (meters["ACTIVE_METER_FLAG"] == "T")]

In [6]:
active.shape

(29313, 45)

In [7]:
# joins active meters to street

joined = gpd.sjoin_nearest(active, unique_streets, distance_col='dist', how='left')
joined.shape

/Users/tessa/opt/anaconda3/envs/stat-222/lib/python3.9/site-packages/geopandas/array.py:344: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


(29334, 63)

In [8]:
joined['street_id'] = joined['Corridor'] + ' ' + joined['Limits']

In [9]:
# how many meters on each street

joined.groupby("street_id").size()

street_id
01st St Clementina St  -  Folsom St               4
01st St Elim St  -  Mission St                    6
01st St Folsom St  -  Guy Pl                     11
01st St Guy Pl  -  Lansing St                    11
01st St Howard St  -  Tehama St                  15
                                                 ..
Wilmot St Webster St  -  Fillmore St              1
Woodacre Dr Ocean Ave  -  Junipero Serra Blvd     1
York St 23rd St  -  24th St                       6
York St 24th St  -  25th St                       7
Zoe St Bryant St  -  Welsh St                     1
Length: 2113, dtype: int64

In [10]:
# transaction data 54 seconds

pay = pd.read_parquet("../data/meter_data_2022.parquet")

In [63]:
pay.head()

,TRANSMISSION_DATETIME,POST_ID,STREET_BLOCK,PAYMENT_TYPE,SESSION_START_DT,SESSION_END_DT,METER_EVENT_TYPE,GROSS_PAID_AMT
45536904,222408714_4_01012022002410,831-02200,EMBARCADERO SOUTH 200,CASH,2022/01/01 12:24:10 AM,2022/01/01 12:24:10 AM,NS,1.50
45536905,222408715_4_01012022003157,546-28060,LEAVENWORTH ST 2800,CASH,2022/01/01 12:31:57 AM,2022/01/01 12:31:57 AM,NS,0.50
45536906,222408723_4_01012022011253,830-03290,EMBARCADERO NORTH 300,CASH,2022/01/01 01:12:53 AM,2022/01/01 01:12:53 AM,NS,0.50
45536907,222408728_4_01012022020117,336-14220,BUSH ST 1400,CASH,2022/01/01 02:01:17 AM,2022/01/01 02:01:17 AM,NS,0.25
45536908,222408729_4_01012022022638,568-05550,MISSION ST 500,CASH,2022/01/01 02:26:38 AM,2022/01/01 02:26:38 AM,NS,1.00


In [11]:
# 36 seconds

df = pd.merge(pay, active, on='POST_ID')

In [66]:
df.head()

,TRANSMISSION_DATETIME,POST_ID,STREET_BLOCK,PAYMENT_TYPE,SESSION_START_DT,SESSION_END_DT,METER_EVENT_TYPE,GROSS_PAID_AMT,OBJECTID,PARKING_SPACE_ID,...,NFC_KEY,SPT_CODE,COLLECTION_ROUTE_DESC,COLLECTION_SUBROUTE_DESC,shape,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods
0,222408714_4_01012022002410,831-02200,EMBARCADERO SOUTH 200,CASH,2022/01/01 12:24:10 AM,2022/01/01 12:24:10 AM,NS,1.50,5272708,0,...,71ME,NaN,South Embarcadero,Embarcadero South/Howard/Steuart | Unit-300/Un...,POINT (-122.39102 37.79161),108,108,1,10,8
1,222412313_4_01022022121822,831-02200,EMBARCADERO SOUTH 200,CREDIT CARD,2022/01/02 12:18:22 PM,2022/01/02 01:18:22 PM,NS,8.00,5272708,0,...,71ME,NaN,South Embarcadero,Embarcadero South/Howard/Steuart | Unit-300/Un...,POINT (-122.39102 37.79161),108,108,1,10,8
2,222412993_4_01022022135210,831-02200,EMBARCADERO SOUTH 200,CREDIT CARD,2022/01/02 01:52:10 PM,2022/01/02 03:52:17 PM,NS,14.49,5272708,0,...,71ME,NaN,South Embarcadero,Embarcadero South/Howard/Steuart | Unit-300/Un...,POINT (-122.39102 37.79161),108,108,1,10,8
3,222448944_4_01032022140304,831-02200,EMBARCADERO SOUTH 200,CREDIT CARD,2022/01/03 02:03:04 PM,2022/01/03 04:23:28 PM,NS,9.06,5272708,0,...,71ME,NaN,South Embarcadero,Embarcadero South/Howard/Steuart | Unit-300/Un...,POINT (-122.39102 37.79161),108,108,1,10,8
4,832929888_9_01032022180700,831-02200,EMBARCADERO SOUTH 200,PAY BY CELL,2022/01/03 06:07:00 PM,2022/01/03 08:07:00 PM,NS,4.00,5272708,0,...,71ME,NaN,South Embarcadero,Embarcadero South/Howard/Steuart | Unit-300/Un...,POINT (-122.39102 37.79161),108,108,1,10,8


In [12]:
# filter out prepaid meters
# 1 minute

df = df[df.SESSION_START_DT != df.SESSION_END_DT]

In [13]:
# change start and end times to datetime
# 92 minutes

df.SESSION_START_DT = pd.to_datetime(df.SESSION_START_DT)
df.SESSION_END_DT = pd.to_datetime(df.SESSION_END_DT)

In [ ]:
dic = dict()
for column in df.columns:
    if column == 'SESSION_START_DT':
        dic[column] = "min"
    elif column == 'SESSION_END_DT':
        dic[column] = "max"
    else:
        dic[column] = "first"

In [ ]:
# merge rows where meter transactions were within 3 mins

def merge_overlapping_rows(df):
    # initialize the group column
    df['group'] = 0
    df = df.sort_values("SESSION_START_DT").reset_index(drop = True)

    # initialize variables for the current group number and end time
    group_num = 0
    prev_end_time = df.loc[0, 'SESSION_END_DT']

    # loop over the rows and assign group numbers
    for i in range(1, len(df)):
        if df.loc[i, 'SESSION_START_DT'] >= (prev_end_time + pd.Timedelta(minutes=3)):
            # new interval, increment group number and update previous end time
            group_num += 1
            prev_end_time = df.loc[i, 'SESSION_END_DT']
            df.loc[i, 'group'] = group_num
        else:
            # overlapping interval, assign the same group number and update previous end time
            df.loc[i, 'group'] = group_num
            prev_end_time = max(prev_end_time, df.loc[i, 'SESSION_END_DT'])

    # group by the group column and aggregate the start and end times
    result = df.groupby('group').agg(dic).reset_index(drop=True)

    return result

In [ ]:
# 50 mins

temp = df.groupby("POST_ID").apply(merge_overlapping_rows)

In [ ]:
# replaces datetime with datetime + 15 mins

def create_interval_endpoint(start_time, td):
    new_time = start_time.replace(hour=0, minute=0, second=0, microsecond=0) + td
    return new_time

In [ ]:
# creates list of 96 intervals, where each element is a column = boolean where True means paid for the entire interval
# 331 minutes

import datetime

interval_series = []


for i in range(0, 24*4):
    start_time_diff = datetime.timedelta(minutes = (i*15))  # time difference of 15 minutes
    end_time_diff = datetime.timedelta(minutes = ((i + 1)*15))  # time difference of 15 minutes
    start_times = temp.SESSION_START_DT.apply(lambda start_time: create_interval_endpoint(start_time, start_time_diff))
    end_times = temp.SESSION_START_DT.apply(lambda start_time: create_interval_endpoint(start_time, end_time_diff))

    interval_series.append((temp['SESSION_START_DT'] <= start_times) & (temp['SESSION_END_DT'] >= end_times))


In [ ]:
int_ser = pd.DataFrame(interval_series)
int_ser.to_csv('../data/intervals2.csv', index=False)
temp.to_csv('../data/temp.csv', index=False)

In [ ]:
# columns: 96 for intervals; session start date, street
session_start_date = temp.SESSION_START_DT.dt.date()
street = temp.street_id
final_df = pd.concat(interval_series + [session_start_date, street], axis=1)

In [ ]:
# checks to see if there is at least one unpaid meter and collapses rows

def count_infractions(df):
    return ~(~df).any()

collapsed_df = final_df.groupby(["session_start_date", "street_id"]).apply(count_infractions).reset_index()

collapsed_df["weekday"] = collapsed_df.session_start_date.dt.day_name()

final_final_df = collapsed_df.groupby("weekday").sum(axis=0)
